# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here

# Scale X_train and X_val using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns)

In [4]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
regression = LinearRegression()
regression.fit(X_train, y_train)

# Calculate a baseline r-squared score on training data and validation
baseline_train = regression.score(X_train, y_train)
baseline_val = regression.score(X_val, y_val)

baseline_train, baseline_val

(0.7868344817421309, 0.6375622643038106)

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [27]:
# Your code here

# Set up data structure
interactions = []

# Find combinations of columns and loop over them
column_combos = list(combinations(X_train.columns, 2))
for (col1, col2) in column_combos:
    # Make copies of X_train and X_val
    X_train_copy = X_train.copy()
    X_val_copy = X_val.copy()
    
    # Add interaction term to data
    X_train_copy["interaction"] = X_train_copy[col1] * X_train_copy[col2]
    X_val_copy["interaction"] = X_val_copy[col1] * X_val_copy[col2]
    
    # Find r-squared score (fit on training data, evaluate on validation data)
    score = LinearRegression().fit(X_train_copy, y_train).score(X_val_copy, y_val)
    
    # Append to data structure
    interactions.append(((col1, col2), score))
    
# Sort and subset the data structure to find the top 7
top7 = sorted(interactions, key=lambda i: i[1], reverse=True)[:7]
print("Top 7 interactions:")
print(top7)

Top 7 interactions:
[(('Fireplaces', 'GrLivArea^2'), 0.8622168119591138), (('TotalBsmtSF', 'LotArea_Fireplaces^3'), 0.8591299955520699), (('1stFlrSF', 'LotArea_Fireplaces'), 0.8514948343960477), (('Fireplaces', 'LotArea_1stFlrSF'), 0.8514948343960476), (('1stFlrSF', 'LotArea_Fireplaces^2'), 0.8510485650398999), (('GarageArea', 'LotArea_TotalBsmtSF'), 0.8430558311784345), (('TotalBsmtSF', 'LotArea_Fireplaces'), 0.8426691385297136)]


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [28]:
# Your code here

# Loop over top 7 interactions
for i in top7:
    # Extract column names from data structure
    col1, col2 = i[0]
    # Construct new column name
    new_col_name = col1 + "_" + col2
    # Add new column to X_train and X_val
    X_train[new_col_name] = X_train[col1] * X_train[col2]
    X_val[new_col_name] = X_val[col1] * X_val[col2]

## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [12]:
# Your code here

# Set up data structure
polynomials = []
# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in (2, 3, 4):
        # Make a copy of X_train and X_val
        X_train_copy = X_train.copy()
        X_val_copy = X_val.copy()
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree, include_bias=False)
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        col_transformed_train = pd.DataFrame(poly.fit_transform(X_train_copy[[col]]))
        col_transformed_val = pd.DataFrame(poly.transform(X_val_copy[[col]]))
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        X_train_copy = pd.concat([X_train_copy.drop(col, axis=1), col_transformed_train], axis=1)
        X_val_copy = pd.concat([X_val_copy.drop(col, axis=1), col_transformed_val], axis=1)
        # Find r-squared score on validation
        score = LinearRegression().fit(X_train_copy, y_train).score(X_val_copy, y_val)

        # Append to data structure
        polynomials.append((col, degree, score))
# Sort and subset the data structure to find the top 7
top7 = sorted(polynomials, key=lambda i: i[-1], reverse=True)[:7]
print("Top 7 polynomials:")
print(top7)

Top 7 polynomials:
[('GrLivArea', 3, 0.8283344365591127), ('OverallQual_2ndFlrSF', 3, 0.8222074062501777), ('LotArea_Fireplaces', 4, 0.8126598088712166), ('LotArea_Fireplaces', 3, 0.8124111133929883), ('OverallQual', 3, 0.8068023565969223), ('OverallQual_2ndFlrSF', 2, 0.805760778692637), ('OverallQual', 4, 0.8033455378866854)]


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [13]:
# Your code here
best_polynomials = pd.DataFrame(top7, columns=["Column", "Degree", "R^2"])
# Filter out duplicates
best_polynomials.drop_duplicates(subset="Column", inplace=True)
# Loop over remaining results
for (col, degree, _) in best_polynomials.values:
    #Create polynomimal terms
    poly = PolynomialFeatures(degree, include_bias=False)
    train_poly = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                              columns=poly.get_feature_names([col]))
    val_poly = pd.DataFrame(poly.transform(X_val[[col]]),
                              columns=poly.get_feature_names([col]))
    
    #Concat back to original
    X_train = pd.concat([X_train.drop(col, axis=1), train_poly], axis=1)
    X_val = pd.concat([X_val.drop(col, axis=1), val_poly], axis=1)

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [14]:
# Your code here
X_train.head()

,LotArea,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,...,OverallQual_2ndFlrSF,OverallQual_2ndFlrSF^2,OverallQual_2ndFlrSF^3,LotArea_Fireplaces,LotArea_Fireplaces^2,LotArea_Fireplaces^3,LotArea_Fireplaces^4,OverallQual,OverallQual^2,OverallQual^3
0,-0.114710,-0.509252,-0.639316,-0.804789,1.261552,0.250689,0.327629,-0.994820,0.092318,0.073336,...,-0.125956,0.015865,-0.001998,0.114116,0.013022,0.001486,1.695855e-04,-0.099842,0.009968,-0.000995
1,-0.176719,-0.509252,0.838208,0.641608,-0.808132,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,...,-0.510770,0.260886,-0.133253,0.175804,0.030907,0.005434,9.552459e-04,0.632038,0.399472,0.252481
2,-0.246336,1.304613,-0.012560,-0.329000,-0.808132,-0.981739,-1.105931,-0.994820,0.081045,0.003094,...,0.672141,0.451774,0.303656,0.245060,0.060055,0.014717,3.606557e-03,-0.831723,0.691762,-0.575354
3,-0.378617,1.304613,-0.339045,-0.609036,-0.808132,-0.981739,-1.134602,0.588023,0.230591,0.128368,...,0.672141,0.451774,0.303656,-0.222636,0.049567,-0.011035,2.456852e-03,-0.831723,0.691762,-0.575354
4,-0.010898,1.304613,-2.531499,-1.315922,0.550523,0.250689,-2.281450,-0.994820,0.014341,0.027589,...,-0.860799,0.740974,-0.637829,0.010842,0.000118,0.000001,1.381725e-08,-1.563603,2.444854,-3.822780


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [15]:
# Your code here
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print("Train R^2:", linreg.score(X_train, y_train))
print("Validation R^2: ", linreg.score(X_val, y_val))

Train R^2: 0.8571817758242435
Validation R^2:  0.6442143449159523


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [17]:
# Your code here
for n in [5, 10, 15, 20, 25]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_val = rfe.transform(X_val)

    linreg = LinearRegression()
    linreg.fit(X_rfe_train, y_train)

    print("Train R^2:", linreg.score(X_rfe_train, y_train))
    print("Test R^2: ", linreg.score(X_rfe_val, y_val))
    print(f"Remaining features: {X_train.shape[1] - n}\n")

Train R^2: 0.776039994126505
Test R^2:  0.6352981725272362
Remaining features: 21

Train R^2: 0.8191862278324272
Test R^2:  0.6743476159860744
Remaining features: 16

Train R^2: 0.8483321237427194
Test R^2:  0.7040137671087112
Remaining features: 11

Train R^2: 0.8495176468836853
Test R^2:  0.7169477986870836
Remaining features: 6

Train R^2: 0.8571732578218183
Test R^2:  0.645929169365526
Remaining features: 1



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [20]:
# Your code here
for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    print("Train R^2:", lasso.score(X_train, y_train))
    print("Validation R^2: ", lasso.score(X_val, y_val))
    print(f"Remaining features: {X_train.shape[1] - (26 - (sum(abs(lasso.coef_) < 10**(-10))))}")
    print(f"alpha: {alpha}\n")

Train R^2: 0.857153074119144
Validation R^2:  0.6485699116355134
Remaining features: 0
alpha: 1

Train R^2: 0.8571373079024015
Validation R^2:  0.6480527180183084
Remaining features: 0
alpha: 10

Train R^2: 0.856958744623801
Validation R^2:  0.6471042867008592
Remaining features: 1
alpha: 100

Train R^2: 0.8506404012942794
Validation R^2:  0.7222278677869787
Remaining features: 9
alpha: 1000

Train R^2: 0.7790529223548714
Validation R^2:  0.7939567393897815
Remaining features: 14
alpha: 10000



Compare the results. Which features would you choose to use?

In [21]:
# Your written answer here

# For RFE the model with the best validation R-Squared had 6 remaining features

# For Lasso the model with the best validation R-Squared used alpha = 10000

# We should choose the features in the Lasso model where alpha = 10000

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [33]:
# Your code here
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, columns=X.columns)

for i in top7:
    print(i)
    col1, col2 = i[0]
    new_col_name = col1 + "_" + col2
    X_test[new_col_name] = X_test[col1] * X_test[col2]

for (col, degree, _) in best_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    poly_test = pd.DataFrame(poly.fit_transform(X_test[[col]]),
                                        columns=poly.get_feature_names([col]))
    X_test = pd.concat([X_test.drop(col, axis=1), poly_test], axis=1)

(('Fireplaces', 'GrLivArea^2'), 0.8622168119591138)


KeyError: 'GrLivArea^2'

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [34]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

print("R-Squared:", final_model.score(X_test, y_test))
print("MSE:", mean_squared_error(y_test, final_model.predict(X_test)))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 33 is different from 10)

## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.